In [3]:
# 代码

import torch
import os

torch.set_printoptions(profile="full")

model = torch.load('/home/zuo/project/xrl/spirl/experiments/hrl/kitchen/cdt_cl_vq_prior_cdt/mkbl_cdtk161_-1+60+6_n+b_s9_1/weights/weights_ep24.pth')

ordered_dict = model['state_dict']['hl_agent']
# keys = ordered_dict.keys()
# for key in keys:
#     print(key)

# 离散化叶子
print(ordered_dict['policy.net.p.0.dc_leaves'].shape)
leaves_tensor = ordered_dict['policy.net.p.0.dc_leaves']
# print(leaves_tensor)
max_indices = torch.max(leaves_tensor, dim=1)[1]
# print(max_indices)
discrete_leaves_tensor = torch.zeros_like(leaves_tensor)
discrete_leaves_tensor.scatter_(1, max_indices.unsqueeze(1), 1)
print(discrete_leaves_tensor.shape)

# 离散化内节点
print(ordered_dict['policy.net.p.0.dc_inner_nodes.weight'].shape)
bias_tensor = ordered_dict['policy.net.p.0.dc_inner_nodes.weight'][:, :1]
inner_tensor = ordered_dict['policy.net.p.0.dc_inner_nodes.weight'][:, 1:]
# print(total_tensor)
max_indices = torch.max(torch.abs(inner_tensor), dim=1)[1]
# print(max_indices)
discrete_inner_tensor = torch.zeros_like(inner_tensor)
discrete_inner_tensor.scatter_(1, max_indices.unsqueeze(1), inner_tensor.gather(1, max_indices.unsqueeze(1)))
total_tensor = torch.cat((bias_tensor, discrete_inner_tensor), 1)
print(total_tensor.shape)

# 更新 ordered_dict 中的张量
ordered_dict['policy.net.p.0.dc_leaves'] = discrete_leaves_tensor
ordered_dict['policy.net.p.0.dc_inner_nodes.weight'] = total_tensor

save_path = '/home/zuo/project/xrl/spirl/experiments/hrl/kitchen/cdt_cl_vq_prior_cdt/discrete_mkbl_cdtk161_-1+60+6_n+b_s9_1/weights/weights_ep24.pth'

dir_path = os.path.dirname(save_path)

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

torch.save(model, save_path)

torch.Size([64, 16])
torch.Size([64, 16])
torch.Size([63, 61])
torch.Size([63, 61])


In [93]:
# 测试

update_model = torch.load('updated_model.pth')
ordered_dict = update_model['state_dict']['hl_agent']
print(ordered_dict['policy.net.p.0.dc_leaves'])
print(ordered_dict['policy.net.p.0.dc_inner_nodes.weight'])

leaves_tensor = torch.tensor([
    [-1.0, 2.0, -3.0, 4.0, -5.0, 6.0, -7.0, 8.0, -9.0, 10.0, -11.0],
    [12.0, -13.0, 14.0, -15.0, 16.0, -17.0, 18.0, -19.0, 20.0, -21.0, 22.0],
    [-23.0, 24.0, -25.0, 26.0, -27.0, 28.0, -29.0, 30.0, -31.0, 32.0, -33.0]
])

total_tensor = torch.tensor([
    [-1.0, 2.0, -3.0, 4.0, -5.0, 6.0, -7.0, 8.0, -9.0, 10.0, -11.0],
    [12.0, -13.0, 14.0, -15.0, 16.0, -17.0, 18.0, -19.0, 20.0, -21.0, 22.0],
    [-23.0, 24.0, -25.0, 26.0, -27.0, 28.0, -29.0, 30.0, -31.0, 32.0, -33.0]
])

# 离散化叶子
# print(ordered_dict['policy.net.p.0.dc_leaves'].shape)
# leaves_tensor = ordered_dict['policy.net.p.0.dc_leaves']
print(leaves_tensor)
max_indices = torch.max(leaves_tensor, dim=1)[1]
print(max_indices)
discrete_leaves_tensor = torch.zeros_like(leaves_tensor)
discrete_leaves_tensor.scatter_(1, max_indices.unsqueeze(1), 1)
print(discrete_leaves_tensor)

# 离散化内节点
print(ordered_dict['policy.net.p.0.dc_inner_nodes.weight'].shape)
bias_tensor = total_tensor[:, :1]
inner_tensor = total_tensor[:, 1:]
print(total_tensor)
max_indices = torch.max(torch.abs(inner_tensor), dim=1)[1]
print(max_indices)
discrete_inner_tensor = torch.zeros_like(inner_tensor)
discrete_inner_tensor.scatter_(1, max_indices.unsqueeze(1), inner_tensor.gather(1, max_indices.unsqueeze(1)))
total_tensor = torch.cat((bias_tensor, discrete_inner_tensor), 1)
print(total_tensor)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         10.,  0.],
        [ 0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 10.,  0.,
          0.,  0.],
        [ 0.,  0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         10.,  0.],
        [ 